# ECG Visualization and Analysis

This notebook provides comprehensive visualizations for ECG signal analysis results.

## Visualizations:
1. Signal quality assessment
2. Feature distributions
3. Model predictions vs actual
4. Confusion matrices
5. ROC curves and performance metrics

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.preprocessing import label_binarize
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully!")

## Generate Sample Data

In [ ]:
# Generate synthetic results for demonstration
np.random.seed(42)

n_samples = 1000
n_classes = 5
class_names = ['Normal', 'PVC', 'PAC', 'VF', 'VT']

# True labels
y_true = np.random.randint(0, n_classes, n_samples)

# Predicted labels (with some errors)
y_pred = y_true.copy()
error_indices = np.random.choice(n_samples, size=int(n_samples * 0.15), replace=False)
y_pred[error_indices] = np.random.randint(0, n_classes, len(error_indices))

# Prediction probabilities
y_proba = np.random.rand(n_samples, n_classes)
y_proba = y_proba / y_proba.sum(axis=1, keepdims=True)

print(f"Generated {n_samples} samples with {n_classes} classes")
print(f"Accuracy: {np.mean(y_true == y_pred):.2%}")

## 1. Confusion Matrix

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
           xticklabels=class_names, yticklabels=class_names,
           cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

# Normalized confusion matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='RdYlGn', 
           xticklabels=class_names, yticklabels=class_names,
           cbar_kws={'label': 'Percentage'})
plt.title('Normalized Confusion Matrix', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

## 2. Classification Report

In [ ]:
# Generate classification report
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
report_df = pd.DataFrame(report).transpose()

print("Classification Report:")
print(report_df)

# Visualize metrics
metrics_df = report_df.iloc[:-3, :3]  # Exclude avg rows and support column

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Precision
axes[0].bar(range(len(metrics_df)), metrics_df['precision'], 
           color='#E63946', alpha=0.7, edgecolor='black')
axes[0].set_xticks(range(len(metrics_df)))
axes[0].set_xticklabels(metrics_df.index, rotation=45)
axes[0].set_title('Precision by Class', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Precision', fontsize=12)
axes[0].set_ylim([0, 1])
axes[0].grid(True, alpha=0.3, axis='y')

# Recall
axes[1].bar(range(len(metrics_df)), metrics_df['recall'], 
           color='#4361EE', alpha=0.7, edgecolor='black')
axes[1].set_xticks(range(len(metrics_df)))
axes[1].set_xticklabels(metrics_df.index, rotation=45)
axes[1].set_title('Recall by Class', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Recall', fontsize=12)
axes[1].set_ylim([0, 1])
axes[1].grid(True, alpha=0.3, axis='y')

# F1-Score
axes[2].bar(range(len(metrics_df)), metrics_df['f1-score'], 
           color='#06A77D', alpha=0.7, edgecolor='black')
axes[2].set_xticks(range(len(metrics_df)))
axes[2].set_xticklabels(metrics_df.index, rotation=45)
axes[2].set_title('F1-Score by Class', fontsize=14, fontweight='bold')
axes[2].set_ylabel('F1-Score', fontsize=12)
axes[2].set_ylim([0, 1])
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 3. ROC Curves

In [ ]:
# Binarize labels for ROC curve
y_true_bin = label_binarize(y_true, classes=range(n_classes))

# Compute ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curves
plt.figure(figsize=(10, 8))
colors = ['#E63946', '#F77F00', '#4361EE', '#06A77D', '#9D4EDD']

for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
            label=f'{class_names[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves for Multi-Class Classification', fontsize=14, fontweight='bold')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print AUC scores
print("\nAUC Scores:")
for i in range(n_classes):
    print(f"  {class_names[i]}: {roc_auc[i]:.4f}")
print(f"  Average: {np.mean(list(roc_auc.values())):.4f}")

## 4. Prediction Confidence Analysis

In [ ]:
# Analyze prediction confidence
max_proba = np.max(y_proba, axis=1)
correct_mask = (y_true == y_pred)

# Plot confidence distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Confidence for correct vs incorrect predictions
axes[0].hist(max_proba[correct_mask], bins=50, alpha=0.7, 
            color='#06A77D', label='Correct', edgecolor='black')
axes[0].hist(max_proba[~correct_mask], bins=50, alpha=0.7, 
            color='#E63946', label='Incorrect', edgecolor='black')
axes[0].set_xlabel('Prediction Confidence', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Prediction Confidence Distribution', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot by class
confidence_by_class = [max_proba[y_true == i] for i in range(n_classes)]
bp = axes[1].boxplot(confidence_by_class, labels=class_names, patch_artist=True)
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
axes[1].set_xlabel('Class', fontsize=12)
axes[1].set_ylabel('Prediction Confidence', fontsize=12)
axes[1].set_title('Confidence by True Class', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print(f"Average confidence (correct): {np.mean(max_proba[correct_mask]):.4f}")
print(f"Average confidence (incorrect): {np.mean(max_proba[~correct_mask]):.4f}")

## 5. Class Distribution Analysis

In [ ]:
# Analyze class distribution
true_counts = np.bincount(y_true)
pred_counts = np.bincount(y_pred)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# True distribution
axes[0].bar(class_names, true_counts, color=colors, alpha=0.7, edgecolor='black')
axes[0].set_title('True Class Distribution', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Count', fontsize=12)
axes[0].grid(True, alpha=0.3, axis='y')
plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=45)

# Predicted distribution
axes[1].bar(class_names, pred_counts, color=colors, alpha=0.7, edgecolor='black')
axes[1].set_title('Predicted Class Distribution', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Count', fontsize=12)
axes[1].grid(True, alpha=0.3, axis='y')
plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.show()

## 6. Error Analysis

In [ ]:
# Analyze misclassifications
misclassified = y_true != y_pred
misclass_df = pd.DataFrame({
    'True': [class_names[i] for i in y_true[misclassified]],
    'Predicted': [class_names[i] for i in y_pred[misclassified]],
    'Confidence': max_proba[misclassified]
})

print(f"Total misclassifications: {len(misclass_df)}")
print("\nMost common misclassifications:")
print(misclass_df.groupby(['True', 'Predicted']).size().sort_values(ascending=False).head(10))

# Visualize error patterns
error_matrix = np.zeros((n_classes, n_classes))
for true, pred in zip(y_true[misclassified], y_pred[misclassified]):
    error_matrix[true, pred] += 1

plt.figure(figsize=(10, 8))
sns.heatmap(error_matrix, annot=True, fmt='.0f', cmap='Reds',
           xticklabels=class_names, yticklabels=class_names,
           cbar_kws={'label': 'Error Count'})
plt.title('Error Pattern Matrix', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:
- Confusion matrix visualization (raw and normalized)
- Classification metrics (precision, recall, F1-score)
- ROC curves and AUC scores for multi-class classification
- Prediction confidence analysis
- Class distribution comparison
- Error pattern analysis

These visualizations provide comprehensive insights into model performance and areas for improvement!